In [ ]:
!pip install nlptutti

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 48.8 MB/s eta 0:00:00


In [ ]:
from transformers import WhisperForConditionalGeneration, WhisperProcessor
import os
import json
import librosa
import torch
import random
import nlptutti as metrics

random.seed(42)

# 모델과 프로세서 로드
model = WhisperForConditionalGeneration.from_pretrained("jwh1449/whisper_base_KoSpeech")
processor = WhisperProcessor.from_pretrained("jwh1449/whisper_base_KoSpeech")
model.generation_config.forced_decoder_ids = None

# 변환할 음성 파일 경로 설정
audio_folder = "/content/drive/MyDrive/STTDATA/데이터모음/KoSpeech원본"
json_folder = "/content/drive/MyDrive/STTDATA/데이터모음/KoSpeech정답"

# 모든 WAV 파일 목록 가져오기
all_files = [f for f in os.listdir(audio_folder) if f.endswith(".wav")]
selected_files = random.sample(all_files, min(100, len(all_files)))

cer_list = []

for file_name in selected_files:
    file_path = os.path.join(audio_folder, file_name)
    json_path = os.path.join(json_folder, file_name.replace(".wav", ".txt"))

    print(f"Processing: {file_name}")

    # 전체 파일 한 번에 로드 (분할 없음)
    audio, sr = librosa.load(file_path, sr=16000)
    input_features = processor(audio, sampling_rate=sr, return_tensors="pt").input_features

    with torch.no_grad():
        generated_ids = model.generate(
            input_features,
            language="ko",
            task="transcribe"
        )

    asr_text = processor.decode(generated_ids[0], skip_special_tokens=True)

    if os.path.exists(json_path):
        with open(json_path, "r", encoding="utf-8") as f:
            data = f.read().strip()

            cer_result = metrics.get_cer(data, asr_text)
            cer = cer_result['cer']
            cer_list.append(cer)
            print(f"CER: {cer:.4f}")
    else:
        print("No reference text found.")

    print(f"Transcription: {asr_text}")
    print(f"Reference: {data}")
    print("-" * 50)

if cer_list:
    print(f"\nAverage CER: {sum(cer_list)/len(cer_list):.4f}")

Processing: KsponSpeech_539357.wav
CER: 0.0833
Transcription: 나는 어제 니 표정 보고 알았어. 걔는 진짜 자기 싫어하는구나.
Reference: 나는 어제 니 표정 보고 알았어. 쟤는 진짜 타기 싫어하는구나.
--------------------------------------------------
Processing: KsponSpeech_608138.wav
CER: 0.0556
Transcription:  그 제목 뭐였지? 우리들의 월급은 정 정당 안 가?
Reference: 그 제목 뭐였지? 우리들의 월급은 정 정당한가?
--------------------------------------------------
Processing: KsponSpeech_618809.wav
CER: 0.0469
Transcription:  그 내 친구의 친구 중에 퍼스널 컬러를 세 번 받아본 애가 있어. 근데 제일 비 비싼대가 있고 그냥 무난한 데가 있단 말이야. 근데 그게 그 차이가 그 그냥 퍼스널 컬러랑
Reference: 그 내 친구에 친구 중에 퍼스널 컬러를 세 번 받아본 애가 있어. 근데 제일 비 비싼 데가 있고 그냥 무난한 데가 있단 말이야. 근데 그게 그 차이가 그냥 퍼스널 컬러랑
--------------------------------------------------
Processing: KsponSpeech_524718.wav
CER: 0.2000
Transcription:  응. 그래서 추천 많이 해서.
Reference: 응. 그거는 추천 많이 해서
--------------------------------------------------
Processing: KsponSpeech_585605.wav
CER: 0.0000
Transcription:  찹쌀탕수육.
Reference: 찹쌀탕수육.
--------------------------------------------------
Processing

In [ ]:
from transformers import WhisperForConditionalGeneration, WhisperProcessor
import os
import json
import librosa
import torch
import random
import nlptutti as metrics

random.seed(42)

# 모델과 프로세서 로드
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-base")
processor = WhisperProcessor.from_pretrained("openai/whisper-base")
model.generation_config.forced_decoder_ids = None

# 변환할 음성 파일 경로 설정
audio_folder = "/content/drive/MyDrive/STTDATA/데이터모음/KoSpeech원본"
json_folder = "/content/drive/MyDrive/STTDATA/데이터모음/KoSpeech정답"

# 모든 WAV 파일 목록 가져오기
all_files = [f for f in os.listdir(audio_folder) if f.endswith(".wav")]
selected_files = random.sample(all_files, min(100, len(all_files)))

cer_list = []

for file_name in selected_files:
    file_path = os.path.join(audio_folder, file_name)
    json_path = os.path.join(json_folder, file_name.replace(".wav", ".txt"))

    print(f"Processing: {file_name}")

    # 전체 파일 한 번에 로드 (분할 없음)
    audio, sr = librosa.load(file_path, sr=16000)
    input_features = processor(audio, sampling_rate=sr, return_tensors="pt").input_features

    with torch.no_grad():
        generated_ids = model.generate(
            input_features,
            language="ko",
            task="transcribe"
        )

    asr_text = processor.decode(generated_ids[0], skip_special_tokens=True)

    if os.path.exists(json_path):
        with open(json_path, "r", encoding="utf-8") as f:
            data = f.read().strip()

            cer_result = metrics.get_cer(data, asr_text)
            cer = cer_result['cer']
            cer_list.append(cer)
            print(f"CER: {cer:.4f}")
    else:
        print("No reference text found.")

    print(f"Transcription: {asr_text}")
    print(f"Reference: {data}")
    print("-" * 50)

if cer_list:
    print(f"\nAverage CER: {sum(cer_list)/len(cer_list):.4f}")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.98k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/290M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.81k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

...

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

Processing: KsponSpeech_539357.wav


You have passed task=transcribe, but also have set `forced_decoder_ids` to [[1, 50259], [2, 50359], [3, 50363]] which creates a conflict. `forced_decoder_ids` will be ignored in favor of task=transcribe.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


CER: 0.2917
Transcription:  나는 어제 네 표정 보고 알았어. 얘는 진짜 다이서라는구나.
Reference: 나는 어제 니 표정 보고 알았어. 쟤는 진짜 타기 싫어하는구나.
--------------------------------------------------
Processing: KsponSpeech_608138.wav
CER: 0.0556
Transcription:  그 제목 뭐였지? 우리들의 월급은 정당한가?
Reference: 그 제목 뭐였지? 우리들의 월급은 정 정당한가?
--------------------------------------------------
Processing: KsponSpeech_618809.wav
CER: 0.1250
Transcription:  내 친구의, 내 친구 중에 퍼스널 컬러를 세 번 받아본 애가 있어. 근데 이제 비싼대가 있고 그냥 무난한 데가 있다 말이야. 근데 그게 그 차이가 그냥 퍼스널 컬러랑
Reference: 그 내 친구에 친구 중에 퍼스널 컬러를 세 번 받아본 애가 있어. 근데 제일 비 비싼 데가 있고 그냥 무난한 데가 있단 말이야. 근데 그게 그 차이가 그냥 퍼스널 컬러랑
--------------------------------------------------
Processing: KsponSpeech_524718.wav
CER: 0.6000
Transcription:  추천만 해서.
Reference: 응. 그거는 추천 많이 해서
--------------------------------------------------
Processing: KsponSpeech_585605.wav
CER: 0.6667
Transcription:  탑사의 탕수요.
Reference: 찹쌀탕수육.
--------------------------------------------------
Processing: KsponSpeech_589709.wav
CER: 0.8889
Transcription

In [ ]:
import requests
import os
import json
import librosa
import torch
import random
import nlptutti as metrics


class ClovaSpeechClient:
    # Clova Speech invoke URL (앱 등록 시 발급받은 Invoke URL)
    invoke_url = 'Your URL'
    # Clova Speech secret key (앱 등록 시 발급받은 Secret Key)
    secret = 'Your KEY'

    def req_url(self, url, completion, callback=None, userdata=None, forbiddens=None, boostings=None, wordAlignment=True, fullText=True, diarization=None, sed=None):
        request_body = {
            'url': url,
            'language': 'ko-KR',
            'completion': completion,
            'callback': callback,
            'userdata': userdata,
            'wordAlignment': wordAlignment,
            'fullText': fullText,
            'forbiddens': forbiddens,
            'boostings': boostings,
            'diarization': diarization,
            'sed': sed,
        }
        headers = {
            'Accept': 'application/json;UTF-8',
            'Content-Type': 'application/json;UTF-8',
            'X-CLOVASPEECH-API-KEY': self.secret
        }
        return requests.post(headers=headers,
                             url=self.invoke_url + '/recognizer/url',
                             data=json.dumps(request_body).encode('UTF-8'))

    def req_object_storage(self, data_key, completion, callback=None, userdata=None, forbiddens=None, boostings=None,
                           wordAlignment=True, fullText=True, diarization=None, sed=None):
        request_body = {
            'dataKey': data_key,
            'language': 'ko-KR',
            'completion': completion,
            'callback': callback,
            'userdata': userdata,
            'wordAlignment': wordAlignment,
            'fullText': fullText,
            'forbiddens': forbiddens,
            'boostings': boostings,
            'diarization': diarization,
            'sed': sed,
        }
        headers = {
            'Accept': 'application/json;UTF-8',
            'Content-Type': 'application/json;UTF-8',
            'X-CLOVASPEECH-API-KEY': self.secret
        }
        return requests.post(headers=headers,
                             url=self.invoke_url + '/recognizer/object-storage',
                             data=json.dumps(request_body).encode('UTF-8'))

    def req_upload(self, file, completion, callback=None, userdata=None, forbiddens=None, boostings=None,
                   wordAlignment=True, fullText=True, diarization=None, sed=None):
        request_body = {
            'language': 'ko-KR',
            'completion': completion,
            'callback': callback,
            'userdata': userdata,
            'wordAlignment': wordAlignment,
            'fullText': fullText,
            'forbiddens': forbiddens,
            'boostings': boostings,
            'diarization': diarization,
            'sed': sed,
        }
        headers = {
            'Accept': 'application/json;UTF-8',
            'X-CLOVASPEECH-API-KEY': self.secret
        }
        # print(json.dumps(request_body, ensure_ascii=False).encode('UTF-8'))
        files = {
            'media': open(file, 'rb'),
            'params': (None, json.dumps(request_body, ensure_ascii=False).encode('UTF-8'), 'application/json')
        }
        response = requests.post(headers=headers, url=self.invoke_url + '/recognizer/upload', files=files)
        return response

if __name__ == '__main__':
    # res = ClovaSpeechClient().req_url(url='http://example.com/media.mp3', completion='sync')
    # res = ClovaSpeechClient().req_object_storage(data_key='data/media.mp3', completion='sync')
    # res = ClovaSpeechClient().req_upload(file='/content/09711-F-84-JA-A-LAR009-2243546.wav', completion='sync')
    # print(res.text)

    random.seed(42)

    # 변환할 음성 파일 경로 설정
    audio_folder = "/content/drive/MyDrive/STTDATA/데이터모음/KoSpeech원본"
    json_folder = "/content/drive/MyDrive/STTDATA/데이터모음/KoSpeech정답"

    # 모든 WAV 파일 목록 가져오기
    all_files = [f for f in os.listdir(audio_folder) if f.endswith(".wav")]
    selected_files = random.sample(all_files, min(100, len(all_files)))

    cer_list = []

    for file_name in selected_files:
      file_path = os.path.join(audio_folder, file_name)
      json_path = os.path.join(json_folder, file_name.replace(".wav", ".txt"))

      print(f"Processing: {file_name}")

      res = ClovaSpeechClient().req_upload(file=file_path, completion='sync')
      res_json = json.loads(res.text)
      asr_text = res_json['text']

      if os.path.exists(json_path):
          with open(json_path, "r", encoding="utf-8") as f:
              data = f.read().strip()

              cer_result = metrics.get_cer(data, asr_text)
              cer = cer_result['cer']
              cer_list.append(cer)
              print(f"CER: {cer:.4f}")
      else:
          print("No reference text found.")

      print(f"Transcription: {asr_text}")
      print(f"Reference: {data}")
      print("-" * 50)

    if cer_list:
        print(f"\nAverage CER: {sum(cer_list)/len(cer_list):.4f}")

Processing: KsponSpeech_539357.wav
CER: 0.0833
Transcription: 나는 어제 네 표정 보고 알았어 쟤는 진짜 사기 싫어하는구나.
Reference: 나는 어제 니 표정 보고 알았어. 쟤는 진짜 타기 싫어하는구나.
--------------------------------------------------
Processing: KsponSpeech_608138.wav
CER: 0.0556
Transcription: 그 제목 뭐였지 우리들의 월급은 정당한가.
Reference: 그 제목 뭐였지? 우리들의 월급은 정 정당한가?
--------------------------------------------------
Processing: KsponSpeech_618809.wav
CER: 0.0476
Transcription: 내 친구의 친구 중에 퍼스널 컬러를 세 번 받아본 애가 있어. 근데 제일 비싼 데가 있고 그냥 무난한 데가 있단 말이야. 근데 그게 그 차이가 그냥 퍼스널 컬러랑.
Reference: 그 내 친구에 친구 중에 퍼스널 컬러를 세 번 받아본 애가 있어. 근데 제일 비 비싼 데가 있고 그냥 무난한 데가 있단 말이야. 근데 그게 그 차이가 그냥 퍼스널 컬러랑
--------------------------------------------------
Processing: KsponSpeech_524718.wav
CER: 0.4000
Transcription: 추천 많이 해서.
Reference: 응. 그거는 추천 많이 해서
--------------------------------------------------
Processing: KsponSpeech_585605.wav
CER: 0.0000
Transcription: 찹쌀탕수육.
Reference: 찹쌀탕수육.
--------------------------------------------------
Processing: KsponSpeech_589709.

KeyError: 'text'

In [ ]:
print("클로바")
print(f"\nAverage CER: {sum(cer_list)/len(cer_list):.4f}")
print(len(cer_list))

클로바

Average CER: 0.1243
68


In [ ]:
!pip install urllib3

In [ ]:
import os
import json
import librosa
import random
import urllib3
import base64
import time
import nlptutti as metrics

random.seed(42)

# 변환할 음성 파일 경로 설정
audio_folder = "/content/drive/MyDrive/STTDATA/데이터모음/KoSpeech원본"
json_folder = "/content/drive/MyDrive/STTDATA/데이터모음/KoSpeech정답"

# 모든 WAV 파일 목록 가져오기
all_files = [f for f in os.listdir(audio_folder) if f.endswith(".wav")]
selected_files = random.sample(all_files, min(100, len(all_files)))

cer_list = []

openApiURL = "http://aiopen.etri.re.kr:8000/WiseASR/Recognition"
accessKey = "Your KEY"
# audioFilePath = "/content/KsponSpeech_622519.wav"
languageCode = "korean"

max_retries = 10  # 최대 10번 재시도

for file_name in selected_files:
    file_path = os.path.join(audio_folder, file_name)
    json_path = os.path.join(json_folder, file_name.replace(".wav", ".txt"))

    print(f"Processing: {file_name}")

    file = open(file_path, "rb")
    audioContents = base64.b64encode(file.read()).decode("utf8")
    file.close()

    requestJson = {
        "argument": {
            "language_code": languageCode,
            "audio": audioContents
        }
    }

    http = urllib3.PoolManager()

    retries = 0
    while retries < max_retries:
        try:
            response = http.request(
                "POST",
                openApiURL,
                headers={"Content-Type": "application/json; charset=UTF-8", "Authorization": accessKey},
                body=json.dumps(requestJson),
                timeout=urllib3.Timeout(connect=5.0, read=30.0)  # 타임아웃 설정
            )
            res_json = json.loads(response.data)
            asr_text = res_json['return_object']['recognized']
            break  # 성공하면 반복 탈출
        except Exception as e:
            print(f"Request failed: {e}. Retrying ({retries+1}/{max_retries})...")
            retries += 1
            time.sleep(5)  # 2초 대기 후 재시도
    else:
        print("Failed after retries. Skipping this file.")
        continue

    if os.path.exists(json_path):
        with open(json_path, "r", encoding="utf-8") as f:
            data = f.read().strip()

            cer_result = metrics.get_cer(data, asr_text)
            cer = cer_result['cer']
            cer_list.append(cer)
            print(f"CER: {cer:.4f}")
    else:
        print("No reference text found.")

    print(f"Transcription: {asr_text}")
    print(f"Reference: {data}")
    print("-" * 50)

if cer_list:
    print(f"\nAverage CER: {sum(cer_list)/len(cer_list):.4f}")


Processing: KsponSpeech_539357.wav
CER: 0.0417
Transcription: 나는 어제 니 표정 보고 알았어. 쟤는 진짜 자기 싫어하는구나.
Reference: 나는 어제 니 표정 보고 알았어. 쟤는 진짜 타기 싫어하는구나.
--------------------------------------------------
Processing: KsponSpeech_608138.wav
CER: 0.1111
Transcription: 그 제목 뭐였지? 우리들에 월급은 정 정당 안 가?
Reference: 그 제목 뭐였지? 우리들의 월급은 정 정당한가?
--------------------------------------------------
Processing: KsponSpeech_618809.wav
CER: 0.0000
Transcription: 그 내 친구에 친구 중에 퍼스널 컬러를 세 번 받아본 애가 있어. 근데 제일 비 비싼 데가 있고 그냥 무난한 데가 있단 말이야. 근데 그게 그 차이가 그냥 퍼스널 컬러랑
Reference: 그 내 친구에 친구 중에 퍼스널 컬러를 세 번 받아본 애가 있어. 근데 제일 비 비싼 데가 있고 그냥 무난한 데가 있단 말이야. 근데 그게 그 차이가 그냥 퍼스널 컬러랑
--------------------------------------------------
Processing: KsponSpeech_524718.wav
CER: 0.2000
Transcription: 응. 그 추천 많이 해서.
Reference: 응. 그거는 추천 많이 해서
--------------------------------------------------
Processing: KsponSpeech_585605.wav
CER: 0.0000
Transcription: 찹쌀탕수육.
Reference: 찹쌀탕수육.
--------------------------------------------------
Processing: Kspon